<a href="https://colab.research.google.com/github/jyotidabass/Image-Processing/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolutional Neural Networks**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset (dummy data)
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess data
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

# Use model for prediction
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
print(f'Predicted classes: {predicted_classes[:10]}')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 37s 23ms/step - accuracy: 0.8937 - loss: 0.3640 - val_accuracy: 0.9753 - val_loss: 0.0832
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.9803 - loss: 0.0643 - val_accuracy: 0.9820 - val_loss: 0.0584
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 45s 25ms/step - accuracy: 0.9876 - loss: 0.0419 - val_accuracy: 0.9828 - val_loss: 0.0578
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - accuracy: 0.9914 - loss: 0.0291 - val_accuracy: 0.9855 - val_loss: 0.0491
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.9940 - loss: 0.0208 - val_accuracy: 0.9821 - val_loss: 0.0652
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9762 - loss: 0.0723
Test accuracy: 0.98
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Predicted classes: [7 2 1 0 4 1 4 9 5 9]


# **Transformers**

In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define a dummy dataset
dummy_data = [
    {'text': 'This is a positive review', 'label': 1},
    {'text': 'This is a negative review', 'label': 0},
    {'text': 'This is a neutral review', 'label': 1}
]

# Preprocess the data
inputs = []
labels = []
for data in dummy_data:
    inputs.append(tokenizer.encode(data['text'], return_tensors='pt', max_length=512, padding='max_length', truncation=True))
    labels.append(torch.tensor(data['label']))

# Create a dataset and data loader
from torch.utils.data import Dataset, DataLoader
class DummyDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx].squeeze(), self.labels[idx]

dataset = DummyDataset(inputs, labels)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids, labels = batch
        input_ids, labels = input_ids.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

# Evaluate the model
model.eval()
with torch.no_grad():
    total_correct = 0
    for batch in data_loader:
        input_ids, labels = batch
        input_ids, labels = input_ids.to(device), labels.to(device)
        outputs = model(input_ids)
        _, predicted = torch.max(outputs.logits, dim=1)
        total_correct += (predicted == labels).sum().item()
    accuracy = total_correct / len(dataset)
    print(f'Accuracy: {accuracy:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1, Loss: 0.6275258660316467
Epoch 2, Loss: 0.6425232291221619
Epoch 3, Loss: 0.6335144639015198
Epoch 4, Loss: 0.6247600317001343
Epoch 5, Loss: 0.6692500710487366
Accuracy: 0.6667


# **Long Short Term Memory (LSTMs)**

In [4]:
# Import necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

# Generate some sample data
np.random.seed(0)
X = np.random.rand(100, 10, 1)
y = np.random.randint(0, 2, 100)

# Scale the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(-1, 10, 1)

# One-hot encode the labels
y = to_categorical(y)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(10, 1)))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_scaled, y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_scaled, y)
print(f'Loss: {loss:.3f}, Accuracy: {accuracy:.3f}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 286ms/step - accuracy: 0.4766 - loss: 0.6941 - val_accuracy: 0.4500 - val_loss: 0.6983
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5586 - loss: 0.6886 - val_accuracy: 0.4500 - val_loss: 0.7092
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5391 - loss: 0.6903 - val_accuracy: 0.4500 - val_loss: 0.7158
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5664 - loss: 0.6856 - val_accuracy: 0.4500 - val_loss: 0.7220
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5820 - loss: 0.6817 - val_accuracy: 0.4500 - val_loss: 0.7250
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5547 - loss: 0.6880 - val_accuracy: 0.4500 - val_loss: 0.7261
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5781 - loss: 0.6818 - val_accuracy: 0.4500 - val_loss: 0.7270
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5547 - loss: 0.6885 - val_accuracy: 0.4500 - val_loss: 0.7238